In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
df_train = pd.read_csv(
        "../Data/genie_datasets/DBN_predictions/all_var_BN_model/cure_ckd_egfr_registry_preprocessed_project_preproc_data_discretized_UCLA_train_with_DBN_predictions.csv",
    )
df_valid = pd.read_csv(
        "../Data/genie_datasets/DBN_predictions/all_var_BN_model/cure_ckd_egfr_registry_preprocessed_project_preproc_data_discretized_UCLA_valid_with_DBN_predictions.csv",
    )
df_test = pd.read_csv(
        "../Data/genie_datasets/DBN_predictions/all_var_BN_model/cure_ckd_egfr_registry_preprocessed_project_preproc_data_discretized_UCLA_test_with_DBN_predictions.csv",
    )
df_Prov = pd.read_csv(
        "../Data/genie_datasets/DBN_predictions/all_var_BN_model/Prov_cure_ckd_egfr_registry_preprocessed_project_preproc_data_discretized_using_UCLA_discritizer_with_DBN_predictions.csv",
    )

/home/panayiotis/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (37,59,81,103,123,125) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/panayiotis/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (37,59,79,81,101,103,110,123,125) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:

#################### static to time zero intra structure

df_intra_static_year0 = pd.read_csv(
    "../Data/simulation_structures/full_model/four_structures_for_simulation_model_csv/chi2_UCLA_intra_struct_yr0_s.csv",
    index_col=0,
)

intra_static_year0_indexes = [
    val.replace("year0", "time_zero").replace("reduction_40_wr", "reduction_40_ge")
    for val in df_intra_static_year0.index
]
df_intra_static_year0.index = intra_static_year0_indexes

cols_intra_static_year0 = [
    col.replace("year0", "time_zero").replace("reduction_40_wr", "reduction_40_ge")
    for col in df_intra_static_year0.columns
]
df_intra_static_year0.columns = cols_intra_static_year0

# filtering out count variables
intra_static_year0_indexes_bool = [
    True if "site" not in val
    # and "count" not in val
    # and "ruca_7" not in val
    # and "coverage" not in val
    # and "uacr" not in val
    # and "upcr" not in val
    else False
    for val in df_intra_static_year0.index
]
cols_intra_static_year0_filt = [
    col.replace("year0", "time_zero").replace("reduction_40_wr", "reduction_40_ge")
    for col in df_intra_static_year0.columns
    if "site" not in col
    # and "count" not in col
    # and "ruca_7" not in col
    # and "coverage" not in col
    # and "uacr" not in col
    # and "upcr" not in col
]

df_intra_static_year0 = df_intra_static_year0.loc[
    intra_static_year0_indexes_bool, cols_intra_static_year0_filt
]

cols_df_intra_static_year0 = list(df_intra_static_year0.columns)

## Missing values

In [23]:
df_miss = pd.DataFrame()

In [24]:
df_miss["UCLA Train dataset % missing values"] = (df_train[cols_df_intra_static_year0].isnull().sum()*100/len(df_train)).round(2)

In [25]:
df_miss["UCLA Valid dataset % missing values"] = (df_valid[cols_df_intra_static_year0].isnull().sum()*100/len(df_valid)).round(2)

In [26]:
df_miss["UCLA Test dataset % missing values"] = (df_test[cols_df_intra_static_year0].isnull().sum()*100/len(df_test)).round(2)

In [27]:
df_miss["PSJH dataset % missing values"] = (df_Prov[cols_df_intra_static_year0].isnull().sum()*100/len(df_Prov)).round(2)

In [30]:
indeces = ["T0 Sr. Creat count", 
           "Year 1 >= 40% eGFR decline",
           "T0 BP count",
           "T0 AV count",
           "T0 Av. eGFR",
           "Study entry CKD from Dx",
           "T0 IPV count",
           "Study entry CKD from eGFR",
           "Study entry DM",
            "T0 Av. HbA1C",
            "Study entry age",
            "Race and ethnicity",
            "T0 days on PPI Med",
            "T0 Av. UACR",
            "T0 Av. DBP",
            "T0 Av. MAP",
            "Study entry CKD from alb pro",
            "T0 UPCR count",
            "TO HbA1C count",
            "T0 Av. SBP",
            "RUCA 7 class",
            "RUCA 4 class",
            "T0 Av. PP",
            "Study entry NSAID Med",
            "Study entry HTN", 
            "Study entry PDM",
            "T0 days on ACEIARB Med",
            "Study entry PPI Med",
            "T0 Av. UPCR",
            "T0 UACR coutn",
            "sex",
            "T0 days on NSAID Med",
            "Study entry GLP1 Med",
            "T0 days on GLP1 Med",
            "T0 days on SGLT2 Med",
            "T0 days on MRA Med",
            "Study entry ACEIARB Med",
            "Study entry SGLT2 Med",
            "Study entry MRA Med",
           ]

In [33]:
df_miss.index = indeces

In [34]:
df_miss

,UCLA Train dataset % missing values,UCLA Valid dataset % missing values,UCLA Test dataset % missing values,PSJH dataset % missing values
T0 Sr. Creat count,0.00,0.00,0.00,0.00
Year 1 >= 40% eGFR decline,0.00,0.00,0.00,0.00
T0 BP count,0.00,0.00,0.00,0.00
T0 AV count,0.00,0.00,0.00,0.00
T0 Av. eGFR,0.14,0.13,0.16,0.12
Study entry CKD from Dx,0.00,0.00,0.00,0.00
T0 IPV count,0.00,0.00,0.00,0.00
Study entry CKD from eGFR,0.00,0.00,0.00,0.00
Study entry DM,0.00,0.00,0.00,0.00
T0 Av. HbA1C,67.96,67.64,67.76,80.71


In [36]:
df_miss.to_csv("../Data/genie_datasets/DBN_predictions/Results/all_vars_model_BN_miss_values.csv")